In [ ]:
import pandas as pd
import numpy as np

In [ ]:
train=pd.io.parsers.read_csv("train.csv")

In [ ]:
test=pd.io.parsers.read_csv("test.csv")

In [ ]:
i1 = pd.DataFrame(test.ID)

In [ ]:
combined_set = pd.concat([train, test], axis = 0)

In [ ]:
combined_set = combined_set.drop('ID', axis = 1)

In [ ]:
combined_set['DOB'] =  pd.to_datetime(combined_set['DOB'],  format='%d/%m/%y',errors='coerce')
combined_set['Lead_Creation_Date'] =  pd.to_datetime(combined_set['Lead_Creation_Date'], format='%d/%m/%y',errors='coerce')

In [ ]:
a = combined_set['DOB']
b = combined_set['Lead_Creation_Date']
age = b -a
age = age.dt.days
age = pd.DataFrame(age)
age.columns = ['age']

In [ ]:
combined_set = pd.concat([combined_set, age], axis=1)
combined_set = combined_set.drop(['DOB','Lead_Creation_Date'], axis =1 )

In [ ]:
for feature in combined_set.columns: 
    if combined_set[feature].dtype == 'object': 
        combined_set[feature] = pd.Categorical(combined_set[feature]).codes 

In [ ]:
train = combined_set.iloc[0:69713]
test = combined_set.iloc[69713:]

In [ ]:
import h2o
h2o.init()
from h2o.automl import H2OAutoML

In [ ]:
train = h2o.H2OFrame(train)
test = h2o.H2OFrame(test)

In [ ]:
x = train.columns
y = "Approved"
x.remove(y)
train[y] = train[y].asfactor()

In [ ]:
aml = H2OAutoML(max_runtime_secs = 7200, nfolds = 3)
aml.train(x = x, y = y,
          training_frame = train)

In [ ]:
lb = aml.leaderboard
lb

In [ ]:
preds = aml.predict(test)

In [ ]:
i2 = preds['predict']
i1.columns = ['ID']
i2 = i2.as_data_frame()
i1.columns = ['Approved']
result = pd.concat([i1, i2], axis=1)
result.to_csv('pred.csv', index= False)